In [2]:
import numpy as np
import pandas as pd
#from numpy.linalg import norm
from scipy import optimize

In [3]:
#Pobranie danych z pliku Excel
df = pd.read_excel('data_heart_disease.xlsx')

In [4]:
#Podział danych na część uczącą i testową (80:20)
df_learn = df.iloc[:int(0.7*len(df)),:]
df_test = df.iloc[int(0.7*len(df)):,:]

In [5]:
X = np.array([df_learn.HighBP,df_learn.HighChol,df_learn.CholCheck,df_learn.BMI,df_learn.Smoker,df_learn.Stroke,df_learn.Diabetes,df_learn.PhysActivity,df_learn.Fruits,df_learn.Veggies,df_learn.HvyAlcoholConsump,df_learn.DiffWalk,df_learn.Sex,df_learn.Age]).transpose()

df_learn_y = df_learn.HeartDiseaseorAttack
y = np.array([df_learn_y]).transpose()

X = X/np.amax(X, axis=0)

In [6]:
X.shape, y.shape

((177576, 14), (177576, 1))

In [7]:
class Neural_Network(object):
    def __init__(self):
        #Define Hyperparameters
        self.inputLayerSize = 14
        self.outputLayerSize = 1
        self.hiddenLayer1Size = 6
        self.hiddenLayer2Size = 6

        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayer1Size)
        self.W2 = np.random.randn(self.hiddenLayer1Size, self.hiddenLayer2Size)
        self.W3 = np.random.randn(self.hiddenLayer2Size, self.outputLayerSize)

    def forward(self, X):
        #Propagate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        self.a3 = self.sigmoid(self.z3)
        self.z4 = np.dot(self.a3, self.W3)
        yHat = self.sigmoid(self.z4)
        return yHat

    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))

    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)

    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J

    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)

        delta4 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z4))
        dJdW3 = np.dot(self.a3.T, delta4)

        delta3 = np.dot(delta4, self.W3.T)*self.sigmoidPrime(self.z3)
        dJdW2 = np.dot(self.a2.T, delta3)

        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)

        return dJdW1, dJdW2, dJdW3

     #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel(), self.W3.ravel()))
        return params

    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayer1Size * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayer1Size))
        W2_end = W1_end + self.hiddenLayer1Size*self.hiddenLayer2Size
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayer1Size, self.hiddenLayer2Size))
        W3_end = W2_end + self.hiddenLayer2Size*self.outputLayerSize
        self.W3 = np.reshape(params[W2_end:W3_end], (self.hiddenLayer2Size, self.outputLayerSize))

    def computeGradients(self, X, y):
        dJdW1, dJdW2, dJdW3 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel(), dJdW3.ravel()))

In [8]:
## Do testu definicji obliczającej gradient z góry - potencjalnie do usunięcia

def computeNumericalGradient(N, X, y):
        paramsInitial = N.getParams()
        numgrad = np.zeros(paramsInitial.shape)
        perturb = np.zeros(paramsInitial.shape)
        e = 1e-4

        for p in range(len(paramsInitial)):
            #Set perturbation vector
            perturb[p] = e
            N.setParams(paramsInitial + perturb)
            loss2 = N.costFunction(X, y)

            N.setParams(paramsInitial - perturb)
            loss1 = N.costFunction(X, y)

            #Compute Numerical Gradient
            numgrad[p] = (loss2 - loss1) / (2*e)

            #Return the value we changed to zero:
            perturb[p] = 0

        #Return Params to original value:
        N.setParams(paramsInitial)

        return numgrad

In [9]:
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N

    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))

    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)

        return cost, grad

    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []

        params0 = self.N.getParams()

        options = {'maxiter': 3000, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS',
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [10]:
NN = Neural_Network()

In [11]:
T = trainer(NN)

In [12]:
T.train(X,y)

C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:30: RuntimeWarning: overflow encountered in square
  return np.exp(-z)/((1+np.exp(-z))**2)
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:26: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:30: RuntimeWarning: overflow encountered in exp
  return np.exp(-z)/((1+np.exp(-z))**2)
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:30: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(-z)/((1+np.exp(-z))**2)
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:26: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:30: RuntimeWarning: overflow encountered in exp
  return np.exp(-z)/((1+np.exp(-z))**2)
C:\Users\pawki\AppData\Local\Temp\ipykernel_24156\2840466036.py:30: RuntimeWarning: overflow encount

         Current function value: 6311.922811
         Iterations: 953
         Function evaluations: 1021
         Gradient evaluations: 1010


In [73]:
NN.forward(X)

array([[0.20604013],
       [0.0122487 ],
       [0.18462831],
       ...,
       [0.0548891 ],
       [0.05313525],
       [0.04211262]])